In [1]:
%pip install -r ../requirements.txt

INFO: pip is looking at multiple versions of azureml-mlflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of azureml-mlflow to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of azureml-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pyopenssl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.6 MB/s eta 0:00:00ta 

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Load environment variables from .env file
# Note: Ensure your .env file is in the root of your user folder in Azure ML
load_dotenv()

SYNAPSE_CONN_STRING = os.getenv("SYNAPSE_CONN_STRING")

# Verify that the connection string was loaded
if not SYNAPSE_CONN_STRING:
    raise ValueError("SYNAPSE_CONN_STRING environment variable not found. Please check your .env file.")

In [3]:
engine = create_engine(SYNAPSE_CONN_STRING, connect_args={'timeout': 60})

query = "SELECT * FROM v_financials_quarterly"

with engine.connect() as connection:
    df = pd.read_sql_query(text(query), connection)

print("Data loaded successfully.")
df.info()

Data loaded successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 12 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   prediction_quarter                          243 non-null    object 
 1   ticker                                      243 non-null    object 
 2   report_date                                 243 non-null    object 
 3   revenues                                    243 non-null    int64  
 4   research_and_development_expense            243 non-null    int64  
 5   selling_general_and_administrative_expense  243 non-null    int64  
 6   net_income                                  243 non-null    int64  
 7   assets                                      243 non-null    int64  
 8   liabilities                                 243 non-null    int64  
 9   gdp                                         243 non-null    f

It is observed that prediction quarter and report_date are objects that need to be converted to dateTime and ticker is also object that needs to be converted to string maybe in future.

In [5]:
df['prediction_quarter']=pd.to_datetime(df['prediction_quarter'])
df['report_date']=pd.to_datetime(df['report_date'])

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
revenues,243.0,3.681009e+10,2.927646e+10,1.384495e+09,1.343500e+10,2.647000e+10,5.575150e+10,1.243000e+11
research_and_development_expense,243.0,3.678233e+09,3.070424e+09,9.577200e+07,1.367000e+09,2.783000e+09,5.430500e+09,1.380800e+10
selling_general_and_administrative_expense,243.0,2.129588e+09,1.697939e+09,1.048510e+08,1.007500e+09,1.425000e+09,2.979000e+09,7.175000e+09
net_income,243.0,9.393598e+09,8.358866e+09,-6.302000e+09,3.059500e+09,6.558000e+09,1.443050e+10,3.633000e+10
assets,243.0,2.035112e+11,1.400548e+11,4.497718e+09,7.113850e+10,1.800980e+11,3.315535e+11,6.190030e+11
liabilities,243.0,1.008991e+11,9.131486e+10,5.438610e+08,2.539850e+10,6.490900e+10,1.741375e+11,3.092590e+11
gdp,243.0,1.931270e+04,4.920852e+03,1.292266e+04,1.498019e+04,1.827978e+04,2.175124e+04,3.048573e+04
cpi,243.0,2.448372e+02,3.381961e+01,1.936667e+02,2.179343e+02,2.374783e+02,2.588177e+02,3.208003e+02
unemployment,243.0,5.783128e+00,2.127740e+00,3.533333e+00,4.133333e+00,4.966667e+00,7.233333e+00,1.300000e+01


In [8]:
df.head()

,prediction_quarter,ticker,report_date,revenues,research_and_development_expense,selling_general_and_administrative_expense,net_income,assets,liabilities,gdp,cpi,unemployment
0,2025-07-01,AAPL,2025-06-30,94036000000,8866000000,6650000000,23434000000,331495000000,265665000000,30485.729,320.800333,4.166667
1,2025-04-01,AAPL,2025-03-31,95359000000,8550000000,6728000000,24780000000,331233000000,264437000000,30042.113,319.492000,4.100000
2,2025-01-01,AAPL,2024-12-31,124300000000,8268000000,7175000000,36330000000,344085000000,277327000000,29825.182,316.538667,4.133333
3,2024-10-01,AAPL,2024-09-30,94930000000,7765000000,6523000000,14736000000,364980000000,308030000000,29511.664,314.182667,4.166667
4,2024-07-01,AAPL,2024-06-30,85777000000,8006000000,6320000000,21448000000,331612000000,264904000000,29147.044,313.095667,4.000000


In [9]:
# CRITICAL: Sort the data by company and then by date.
# This is essential for all time-series operations.
df.sort_values(by=['ticker', 'prediction_quarter'], inplace=True)
df.reset_index(drop=True, inplace=True)

print("Data sorted and prepared.")
df.head()

Data sorted and prepared.


,prediction_quarter,ticker,report_date,revenues,research_and_development_expense,selling_general_and_administrative_expense,net_income,assets,liabilities,gdp,cpi,unemployment
0,2005-07-01,AAPL,2005-06-30,3520000000,145000000,472000000,319000000,10488000000,3667000000,12922.656,193.666667,5.100000
1,2005-10-01,AAPL,2005-09-30,3678000000,147000000,470000000,430000000,11551000000,4085000000,13142.642,196.600000,4.966667
2,2006-01-01,AAPL,2005-12-31,5749000000,182000000,632000000,565000000,14181000000,5801000000,13324.204,198.433333,4.966667
3,2006-04-01,AAPL,2006-03-31,4359000000,176000000,592000000,410000000,13911000000,5229000000,13599.160,199.466667,4.733333
4,2006-07-01,AAPL,2006-06-30,4370000000,175000000,584000000,472000000,15114000000,5784000000,13753.424,201.266667,4.633333


In [10]:
# --- Create the Target Variable (y) ---
# The goal is to predict the *next* quarter's revenue.
df['target_revenue_next_q'] = df.groupby('ticker')['revenues'].shift(-1)

# --- Create Time-Based Features (X) ---
df['quarter'] = df['prediction_quarter'].dt.quarter
df['year'] = df['prediction_quarter'].dt.year

# --- Create Lag and Rolling Features (X) ---
# This loop will systematically create all the features you requested.

features_to_engineer = {
    'revenues': {
        'lags': [1, 2, 3, 4], # ✅ ADDED LAG 4 FOR YEAR-OVER-YEAR COMPARISON
        'rolling_avg': [4]   # ✅ WILL BE USED FOR SEASONAL DIFFERENCE
    },
    'net_income': {'lags': [2, 3, 4], 'rolling_avg': [4]},
    'research_and_development_expense': {'lags': [4, 5, 6, 7, 8]},
    'selling_general_and_administrative_expense': {'lags': [1, 2], 'rolling_avg': [4]},
    'assets': {'lags': [2, 3, 4]},
    'liabilities': {'lags': [2, 3, 4]},
    'gdp': {'lags': [1, 2]},
    'cpi': {'lags': [1]},
    'unemployment': {'lags': [1, 2]}
}

for col, params in features_to_engineer.items():
    if 'lags' in params:
        for lag in params['lags']:
            # Group by ticker to ensure lags are calculated per company
            df[f'{col}_lag_{lag}'] = df.groupby('ticker')[col].shift(lag)
    if 'rolling_avg' in params:
        for window in params['rolling_avg']:
            # Calculate rolling average on a per-company basis
            df[f'{col}_rolling_avg_{window}'] = df.groupby('ticker')[col].rolling(window=window).mean().reset_index(0,drop=True)

# --- ✅ CREATE THE SEASONAL DIFFERENCE FEATURE ---
# This feature isolates how much a quarter's revenue deviates from its yearly trend.
df['revenue_seasonal_diff'] = df['revenues'] - df['revenues_rolling_avg_4']

print("Advanced feature engineering complete.")
df.tail()

Advanced feature engineering complete.


,prediction_quarter,ticker,report_date,revenues,research_and_development_expense,selling_general_and_administrative_expense,net_income,assets,liabilities,gdp,...,assets_lag_4,liabilities_lag_2,liabilities_lag_3,liabilities_lag_4,gdp_lag_1,gdp_lag_2,cpi_lag_1,unemployment_lag_1,unemployment_lag_2,revenue_seasonal_diff
238,2024-07-01,MSFT,2024-06-30,64727000000,8056000000,2246000000,22036000000,512163000000,243686000000,29147.044,...,4.119760e+11,2.322900e+11,2.250710e+11,2.057530e+11,28708.161,28424.722,310.974333,3.833333,3.800000,3.446500e+09
239,2024-10-01,MSFT,2024-09-30,65585000000,7544000000,1673000000,24667000000,523013000000,235290000000,29511.664,...,4.457850e+11,2.311230e+11,2.322900e+11,2.250710e+11,29147.044,28708.161,313.095667,4.000000,3.833333,2.037500e+09
240,2025-01-01,MSFT,2024-12-31,69632000000,7917000000,1823000000,24108000000,533898000000,231203000000,29825.182,...,4.705580e+11,2.436860e+11,2.311230e+11,2.322900e+11,29511.664,29147.044,314.182667,4.166667,4.000000,4.181500e+09
241,2025-04-01,MSFT,2025-03-31,70066000000,8198000000,1737000000,25824000000,562624000000,240733000000,30042.113,...,4.842750e+11,2.352900e+11,2.436860e+11,2.311230e+11,29825.182,29511.664,316.538667,4.133333,4.166667,2.563500e+09
242,2025-07-01,MSFT,2025-06-30,76441000000,8829000000,1990000000,27233000000,619003000000,275524000000,30485.729,...,5.121630e+11,2.312030e+11,2.352900e+11,2.436860e+11,30042.113,29825.182,319.492000,4.100000,4.133333,6.010000e+09


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 44 columns):
 #   Column                                                    Non-Null Count  Dtype         
---  ------                                                    --------------  -----         
 0   prediction_quarter                                        243 non-null    datetime64[ns]
 1   ticker                                                    243 non-null    object        
 2   report_date                                               243 non-null    datetime64[ns]
 3   revenues                                                  243 non-null    int64         
 4   research_and_development_expense                          243 non-null    int64         
 5   selling_general_and_administrative_expense                243 non-null    int64         
 6   net_income                                                243 non-null    int64         
 7   assets                                      